# linking

> WRDS Concordances

Since WRDS is a subscription service, in the documentation below, I can not show any of the data that is being retrieved/generated (will show only column names).

In [ ]:
#| default_exp wrds.linking

In [ ]:
#| exports
import pandas as pd

import pandasmore as pdm
from finsets.wrds import wrds_api

## Merging CRSP and COMPUSTAT

Official documentation for this merge is [here](https://wrds-www.wharton.upenn.edu/pages/wrds-research/applications/linking-databases/linking-crsp-and-compustat/)

Relevant information from that site:

- Because TICKERs and CUSIPs change over time, most datasets use permanent identifiers to indicate the same securities and companies. For example, CRSP employs PERMNO to track stocks, Compustat uses GVKEY to follow companies, and a combination of GVKEY and IID is used to track securities. As indicated in the preceding table, both PERMNO and GVKEY (+IID) remain the same, regardless of changes in TICKER, CUSIP, and company names.

- The CRSP/Compustat Merged Database (CCM) is comprised of CRSP and Compustat data, together with the link and link-history references between these two databases. The key product of CCM is a permanent identifier linking table (PERMNO to GVKEY+IID), though CCM is often wrongly mistaken as the merged product of CRSP stock market data with Compustat accounting data. From an end user’s perspective, CCM only adds a link of PERMNO and PERMCO to the Compustat database, so that Compustat items can be searched by CRSP identifiers.


LinkType: A code describing the connection between the CRSP and Compustat data.

- LU: Link research complete. Standard connection between databases.
- LC: Non-researched link to issue by CUSIP.
- LD: Duplicate link to a security. Another GVKEY/IID is a better link to that CRSP record.

LinkPrim: Primary issue indicator for the link.

- P: Primary, identified by Compustat in monthly security data.
- C: Primary, assigned by CRSP to resolve ranges of overlapping or missing primary markers from Compustat in order to produce one primary security throughout the company history.

LINKDT: The first effective date of the link.

LINKENDDT: The last date when the link is valid. For a currently valid link, LINKENDDT is set to a NULL value (.E in SAS format).

In [ ]:
#| export
def gvkey_permno_m(nrows: int=None, #Number of rows to download. If None, full dataset will be downloaded
                   ) -> pd.DataFrame:
    """CRSP Monthly ids, with gvkeys"""

    limit_clause = f"LIMIT {nrows}" if nrows is not None else ""
    sql_string=f"""SELECT a.date, a.permno, c.gvkey
                  FROM crsp.msf a
                  INNER JOIN crsp.msenames b ON a.permno = b.permno
                                             AND a.date BETWEEN b.namedt AND b.nameendt 
                  INNER JOIN crsp.ccmxpf_lnkhist c ON a.permno = c.lpermno 
                                                   AND c.linktype IN ('LU','LC') AND c.linkprim IN ('P','C')
                                                   AND a.date BETWEEN c.linkdt AND COALESCE(c.linkenddt, CURRENT_DATE)
                    {limit_clause}                                   
                """
    
    df = wrds_api.download(sql_string)
    df = pdm.setup_panel(df, panel_ids='permno', time_var='date', freq='M',
                         drop_index_duplicates=True, duplicates_which_keep='last')
    df['gvkey'] = df['gvkey'].astype('string')
    return df.reset_index()[['permno','Mdate','gvkey']].copy()  

In [ ]:
#| eval: false
df = gvkey_permno_m(nrows=1)

Loading library list...
Done


In [ ]:
#| eval: false
df

,permno,Mdate,gvkey
0,10000,1986-01,013007


In [ ]:
#| export
def gvkey_permno_a(nrows: int=None, #Number of rows to download. If None, full dataset will be downloaded
                   ) -> pd.DataFrame:
    """qvkey to permno correspondence at the annual frequency. As done by CCM."""

    limit_clause = f"LIMIT {nrows}" if nrows is not None else ""
    sql_string=f"""SELECT a.datadate, a.gvkey , b.lpermno as permno
                    FROM comp.funda a
                    INNER JOIN crsp.ccmxpf_lnkhist  b ON a.gvkey = b.gvkey
                    WHERE datadate BETWEEN b.linkdt AND COALESCE(b.linkenddt, CURRENT_DATE)
                            AND b.linktype IN ('LU','LC') AND b.linkprim IN ('P','C')
                            AND indfmt='INDL' AND datafmt='STD' AND popsrc='D' AND consol='C'
                    {limit_clause}            
                """
    
    df = wrds_api.download(sql_string)
    df = pdm.setup_panel(df, panel_ids='permno', time_var='datadate', freq='Y',
                         drop_index_duplicates=True, duplicates_which_keep='last')
    df['gvkey'] = df['gvkey'].astype('string')
    return df.reset_index()[['permno','Ydate','gvkey']].copy()  

In [ ]:
#| eval: false
df = gvkey_permno_a(nrows=1)

Loading library list...
Done


In [ ]:
#| eval: false
df

,permno,Ydate,gvkey
0,25881,1970,001000


In [ ]:
#| export
def gvkey_permno_q(nrows : int=None, #Number of rows to download. If None, full dataset will be downloaded
                   ) -> pd.DataFrame:
    """qvkey to permno correspondence at the quarterly frequency. As done by CCM."""
    
    limit_clause = f"LIMIT {nrows}" if nrows is not None else ""
    sql_string=f"""SELECT a.datadate, a.gvkey , b.lpermno as permno
                    FROM comp.fundq a
                    INNER JOIN crsp.ccmxpf_lnkhist  b ON a.gvkey = b.gvkey
                    WHERE datadate BETWEEN b.linkdt AND COALESCE(b.linkenddt, CURRENT_DATE)
                            AND b.linktype IN ('LU','LC') AND b.linkprim IN ('P','C')
                            AND indfmt='INDL' AND datafmt='STD' AND popsrc='D' AND consol='C'            
                    {limit_clause}
                """
    
    df = wrds_api.download(sql_string)
    df = pdm.setup_panel(df, panel_ids='permno', time_var='datadate', freq='Q',
                         drop_index_duplicates=True, duplicates_which_keep='last')
    df['gvkey'] = df['gvkey'].astype('string')
    return df.reset_index()[['permno','Qdate','gvkey']].copy()    

In [ ]:
#| eval: false
df = gvkey_permno_q(nrows=1)

Loading library list...
Done


In [ ]:
#| eval: false
df

,permno,Qdate,gvkey
0,25881,1970Q4,001000


## Merging IBES and CRSP

In [ ]:
#| export
def ibes_ticker_permno(nrows: int=None):
    limit_clause = ''
    if nrows is not None: limit_clause = f' LIMIT {nrows}'
    return wrds_api.download('SELECT * FROM wrdsapps_link_crsp_ibes.ibcrsphist' + limit_clause)

In [ ]:
#| eval: false
ibes_permnos = ibes_ticker_permno(nrows=1)

Loading library list...
Done


In [ ]:
#| eval: false
ibes_permnos

,ticker,permno,ncusip,sdate,edate,score
0,0000,14471,87482X10,2014-02-20,2016-08-31,1.0


## Merging TRACE Bond Data with CRSP

In [ ]:
#| export 
def bond_cusip_permno(nrows: int=None):
    limit_clause = ''
    if nrows is not None: limit_clause = f' LIMIT {nrows}'
    return wrds_api.download('SELECT * FROM wrdsapps.bondcrsp_link' + limit_clause)

In [ ]:
#| eval: false
cusip_permnos = bond_cusip_permno(nrows=1)

Loading library list...
Done


In [ ]:
#| eval: false
cusip_permnos

,cusip,permno,permco,trace_startdt,trace_enddt,crsp_startdt,crsp_enddt,link_startdt,link_enddt
0,000336AC1,60687,21372,2014-08-22,2014-08-22,2002-01-02,2024-12-31,2014-08-22,2014-08-22


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()